In [2]:
import sys
sys.path.append('..')
sys.path.append('/Users/jh7685/Documents/Projects/pysurfer')
from pysurfer.mgz_helper import map_values_as_mgz
import os
import pandas as pd
import numpy as np
from sfp_nsdsyn import *
import warnings
from itertools import product
warnings.filterwarnings("ignore", category=UserWarning)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
nsd_dir ='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset'
output_dir='/Volumes/server/Projects/sfp_nsd/derivatives'
#freesurfer_dir='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/nsddata/freesurfer'

In [6]:
subj='subj01'
hemi='lh'
betas = os.path.join(nsd_dir, 'nsddata_betas', 'ppdata', f'{subj}', 'nativesurface', 'nsdsyntheticbetas_fithrf_GLMdenoise_RR', f'{hemi}.betas_nsdsynthetic.hdf5')
design_mat=os.path.join(nsd_dir, 'nsddata', 'experiments', 'nsdsynthetic', 'nsdsynthetic_expdesign.mat')
task_keys=['fixation_task','memory_task']
stim_info=os.path.join(nsd_dir, 'nsdsyn_stim_description.csv')
task_average=True
eccentricity_path= os.path.join(nsd_dir, 'nsddata', 'freesurfer', f'{subj}', 'label',  f'{hemi}.prfeccentricity.mgz')

# Load whole brain betas

In [7]:
betas_df = sfm.get_whole_brain_betas(betas_path=betas, design_mat_path=design_mat,
                          stim_info_path=stim_info,
                          task_keys=task_keys, task_average=False, eccentricity_path=eccentricity_path,
                          x_axis='voxel', y_axis='stim_idx', long_format=True)

In [ ]:
bts.get_sigma_v_for_whole_brain(betas_df)

In [ ]:
design_mat=os.path.join(nsd_dir, 'nsddata', 'experiments', 'nsdsynthetic', 'nsdsynthetic_expdesign.mat')
task_keys=['fixation_task','memory_task']
stim_info=os.path.join(nsd_dir, 'nsdsyn_stim_description.csv')
hemi_list= ['lh','rh']
subj_list = [utils.sub_number_to_string(sn, dataset="nsdsyn") for sn in np.arange(1,9)]
for subj, hemi in product(subj_list[1:], hemi_list):
    betas = os.path.join(nsd_dir, 'nsddata_betas', 'ppdata', f'{subj}', 'nativesurface', 'nsdsyntheticbetas_fithrf_GLMdenoise_RR', f'{hemi}.betas_nsdsynthetic.hdf5')
    eccentricity_path= os.path.join(nsd_dir, 'nsddata', 'freesurfer', f'{subj}', 'label',  f'{hemi}.prfeccentricity.mgz')
    betas_df = sfm.get_whole_brain_betas(betas_path=betas, design_mat_path=design_mat,
                              stim_info_path=stim_info,
                              task_keys=task_keys, task_average=False,
                              eccentricity_path=eccentricity_path,
                              x_axis='voxel', y_axis='stim_idx', long_format=True)
    betas_df = betas_df.query('~names.str.contains("intermediate").values')
    w = get_precision_val_map(betas_df)
    print(f'**{subj}-{hemi}')
    map_values_as_mgz(eccentricity_path, w, save_path=f'/Volumes/server/Projects/sfp_nsd/derivatives/sfp_maps/mgzs/nsdsyn/{hemi}.sub-{subj}_value-precision.mgz')